In [149]:
import os
import pandas as pd

In [150]:
df = pd.read_csv('Data/spotify_millsongdata.csv')

In [151]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [152]:
df.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [153]:
df.shape

(57650, 4)

In [154]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [155]:
df = df.drop('link', axis=1, errors='ignore').reset_index(drop=True)


In [156]:
df.head()

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [157]:
df["text"][0]

"Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [158]:
df=df.sample(5000)
df.shape

(5000, 3)

In [159]:
df['text']=df['text'] = df['text'].str.lower().str.replace(r'^\w\s', ' ').replace(r'\n', '', regex=True)

In [160]:
df.head()

,artist,song,text
29775,Diana Ross,I Love You (That's All That Really Matters),"(aretha franklin) \ri love you, i love you \..."
26743,Carly Simon,Anyone But Me,straight from the hall of fame \rone of the w...
24065,Andrea Bocelli,September Morn,written by: neil diamond and gilbert becaud \...
22030,Yes,Universal Garden,looking up into the endless sky \rstaring int...
18854,Steve Miller Band,The Gangster Is Back,"look out, the gangster's back \ri done traded..."


In [3]:
import nltk
from nltk.stem.porter import PorterStemmer

In [4]:
stemmer = PorterStemmer()

In [ ]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(i) for i in token]
    return "".join(a)